In [1]:
print("all_ok")

all_ok


In [2]:
from langchain_groq import ChatGroq

c:\LLMOPS\Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [122]:
llm=ChatGroq(model="qwen/qwen3-32b")

In [123]:
llm.invoke("What is the capital of France?").content

'<think>\nOkay, so the question is asking for the capital of France. Hmm, let me think. I remember that France is a country in Europe, right? I think the capital is a city that starts with "P." Wait, Paris? Yeah, that sounds familiar. I\'ve heard of Paris being called the City of Light or something like that. But wait, is it definitely Paris? Maybe I\'m mixing it up with another city. Let me check in my mind. Other major cities in France include Lyon, Marseille, Bordeaux, and maybe Toulouse. But the capital is almost certainly Paris. I think the French government is based there, with the Eiffel Tower and the Louvre. Yeah, that\'s Paris. I don\'t think there\'s any other city that\'s the capital. Wait, was there ever a different capital before? I don\'t recall. I think it\'s always been Paris. So yeah, the answer should be Paris.\n</think>\n\nThe capital of France is **Paris**. It is not only the political and administrative center of the country but also a major cultural, economic, and

1.Data ingestion

In [8]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
import os

In [11]:
os.getcwd()

'c:\\LLMOPS\\Project\\notebook'

In [12]:
file_path=os.path.join(os.getcwd(),"data","sample.pdf")

In [13]:
loader=PyPDFLoader(file_path)

In [18]:
documents=loader.load()

In [19]:
documents[0].metadata

{'producer': 'PyPDF2',
 'creator': 'PyPDF',
 'creationdate': '',
 'subject': 'Neural Information Processing Systems http://nips.cc/',
 'publisher': 'Curran Associates, Inc.',
 'language': 'en-US',
 'created': '2017',
 'eventtype': 'Poster',
 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On English-to-

In [20]:
len(documents)

11

In [37]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=200,length_function=len)

In [51]:
docs=text_splitter.split_documents(documents)

In [52]:
len(docs)

42

In [53]:
docs[2].page_content

'efforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [31, 21, 13].\n∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started\nthe effort to evaluate this idea. Ashish, with Illia, designed and implemented the ﬁrst Transformer models and\nhas been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and\nefﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, greatly improving result

In [54]:
from langchain_community.vectorstores import FAISS

In [56]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [57]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 392.59it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
embedding_model.embed_query(docs[0].page_content)

In [60]:
len(embedding_model.embed_query(docs[0].page_content))

384

In [61]:
vector_store=FAISS.from_documents(docs,embedding_model)

In [66]:
relevant_docs=vector_store.similarity_search("what is self attention mechanism?",k=5)

In [67]:
retriever=vector_store.as_retriever()

In [109]:
prompt_template="""Use the following pieces of context to answer the question at the end. If you don't know the answer, say you don't know. {context} Question: {question} Answer:"""

In [110]:
from langchain_core.prompts import PromptTemplate

In [111]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [112]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, say you don't know. {context} Question: {question} Answer:")

In [114]:
from langchain_core.output_parsers import StrOutputParser

In [115]:
parser=StrOutputParser()

In [116]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [117]:
from langchain_core.runnables import RunnablePassthrough

In [126]:
rag_chain=((({"context":retriever|format_docs,"question":RunnablePassthrough()}
   |prompt
   |llm
   |StrOutputParser())))

In [127]:
rag_chain.invoke("what is self attention mechanism?")

"<think>\nOkay, so the user is asking about the self-attention mechanism. Let me start by recalling what I read in the provided context. The first paragraph mentions that self-attention, also called intra-attention, is an attention mechanism that relates different positions of a single sequence to compute a representation. It's used in tasks like reading comprehension and summarization.\n\nThen, there's a comparison with end-to-end memory networks that use recurrent attention. The Transformer is highlighted as the first model relying entirely on self-attention without RNNs or convolution. The key point is that self-attention allows the model to relate different positions in the sequence without sequential computation, which is a big deal because it enables parallel processing. \n\nThe context also talks about how in models like ConvS2S, the number of operations increases with the distance between positions, making distant dependencies harder to learn. The Transformer reduces this to a 